In [1]:
!pip install --upgrade tensorflow==2.18.0
!pip install --upgrade numpy==1.26.4
!pip install  --upgrade tensorflow_probability==0.25.0
!pip install --upgrade keras==3.7.0
# !pip install pyDOE
!pip install pyDOE

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 73.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.0
    Uninstalling tensorboard-2.17.0:
      Successfully uninstalled tensorboard-2.17.0
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Uninstalling keras-3.4.1:
      Successfully uninstalled keras-3.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.0
    Uninstalling tensorflow-2.17.0:
      Successfully uninstalled tensorflow-2.17.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.10.0 requires tensorflow==2.17.0, but you have tensorflow 2.18.0 wh

In [2]:
 #!pip install  --upgrade tensorflow_probability==0.25.0

In [3]:
import sys
sys.path.insert(0, '../../Utilities/')

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from pyDOE import lhs
'from plotting import newfig, savefig'
from mpl_toolkits.mplot3d import Axes3D
import time
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from keras import models 

In [4]:
zzz =tf.keras.models
zz = tf.keras.models.Model

In [5]:
#kernel_initializer='glorot_normal'

In [6]:
!which python

/usr/local/bin/python


/usr/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


!dir

In [7]:
!conda install python

/bin/bash: line 1: conda: command not found


In [8]:
!which python

/usr/local/bin/python


In [9]:
!ln 


ln: missing file operand
Try 'ln --help' for more information.


In [10]:
noise = 0.0        
layers = [2, 100, 100, 100, 100, 2]
    

# Doman bounds
lb = np.array([-5.0, 0.0])
ub = np.array([5.0, np.pi/2])

N0 = 50
N_b = 50
N_f = 20000

data = scipy.io.loadmat('/kaggle/input/shroed/NLS.mat')

t = data['tt'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = data['uu']
Exact_u = np.real(Exact)
Exact_v = np.imag(Exact)
Exact_h = np.sqrt(Exact_u**2 + Exact_v**2)

X, T = np.meshgrid(x,t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact_u.T.flatten()[:,None]
v_star = Exact_v.T.flatten()[:,None]
h_star = Exact_h.T.flatten()[:,None]

###########################
np.random.seed(1234)

idx_x = np.random.choice(x.shape[0], N0, replace=False)
x0 = x[idx_x,:]
u0 = Exact_u[idx_x,0:1]
v0 = Exact_v[idx_x,0:1]

idx_t = np.random.choice(t.shape[0], N_b, replace=False)
tb = t[idx_t,:]

X_f = lb + (ub-lb)*lhs(2, N_f)
#X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
#X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
#X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)

In [11]:
def parameters( x0, u0, v0, tb, X_f, lb, ub):
       
       X0 = np.concatenate((x0, 0*x0), 1) # (x0, 0)
       X_lb = np.concatenate((0*tb + lb[0], tb), 1) # (lb[0], tb)
       X_ub = np.concatenate((0*tb + ub[0], tb), 1) # (ub[0], tb)
       
              
       x0 = X0[:,0:1]
       t0 = X0[:,1:2]

       x_lb = X_lb[:,0:1]
       t_lb = X_lb[:,1:2]

       x_ub = X_ub[:,0:1]
       t_ub = X_ub[:,1:2]
       
       x_f = X_f[:,0:1]
       t_f = X_f[:,1:2]
       
       u0 = u0
       v0 = v0
       
       q = tf.constant(x0,tf.float32)
       t0_tf = tf.constant(t0,tf.float32)

       u0_tf = tf.constant(u0,tf.float32)
       v0_tf = tf.constant(v0,tf.float32)

       x_lb_tf = tf.constant(x_lb,tf.float32)
       t_lb_tf = tf.constant(t_lb,tf.float32)

       x_ub_tf = tf.constant(x_ub,tf.float32)
       t_ub_tf = tf.constant(t_ub,tf.float32)

       x_f_tf = tf.constant(x_f,tf.float32)
       t_f_tf = tf.constant(t_f,tf.float32)
       return(q,t0_tf,u0_tf,v0_tf,x_lb_tf,t_lb_tf,x_ub_tf,t_ub_tf,x_f_tf,t_f_tf)
       # Initialize NNs
q,t0_tf,u0_tf,v0_tf,x_lb_tf,t_lb_tf,x_ub_tf,t_ub_tf,x_f_tf,t_f_tf = parameters(x0, u0, v0, tb, X_f, lb, ub)

In [12]:
#kein neues netz
def initialize_NN( ebenen):        
       components = []
       biases = []
       c0 = tf.keras.Input(shape=(ebenen[0],))
       components.append(c0)  
       num_ebenen = len(ebenen) 
       for l in range(1,num_ebenen):
              c =  tf.keras.layers.Dense( (ebenen[l]), activation='tanh')
              components.append(c)       
       return components
def NN_manual():        
       
       biases = []
       z1 = zzz.Sequential([
        tf.keras.Input(shape=(2,)),
        tf.keras.layers.Dense(100, activation='tanh'),
        tf.keras.layers.Dense(100, activation='tanh'),
        tf.keras.layers.Dense(100, activation='tanh'),
        tf.keras.layers.Dense(100, activation='tanh'),
        tf.keras.layers.Dense( 2)
       ])
       return z1

def neural_net(components):
       
       I = components[0]
       num_ebenen =  len(components)
       for l in range(1,num_ebenen -1):
              I = components[l](I)
       
       output_tensor = components[(num_ebenen-1)](I)
       return zz(components[0],output_tensor)
components = initialize_NN(layers)
nn = neural_net(components)# außerhalb?

In [13]:
l0 = tf.keras.layers.Input(shape=(2,))
l = tf.keras.layers.Dense(100, activation='tanh')(l0)
l = tf.keras.layers.Dense(100, activation='tanh')(l)
l = tf.keras.layers.Dense(100, activation='tanh')(l)
l = tf.keras.layers.Dense(100, activation='tanh')(l)
l = tf.keras.layers.Dense( 2)(l) #error
t = zz(l0,l)


In [14]:

def net(x,t):
    uv =  nn(tf.concat([x,t],1))
    u = uv[:,0:1]
    v = uv[:,1:2]
    return u,v

In [15]:
def net_uv( x, t):
       with tf.GradientTape(persistent=True) as tape:
              tape.watch(x)
              tape.watch(t)
              uv = nn( tf.concat([x,t],1))
              u = uv[:,0:1]
              v = uv[:,1:2]
              # tf.print(tape.gradient(u,x))
              u_x = tape.gradient(u, x)[0]
              v_x = tape.gradient(v, x)[0]
       return u, v, u_x, v_x
def net_f_uv(x, t):
    with tf.GradientTape(persistent=True) as tape:
       tape.watch(x)
       tape.watch(t)
       u, v, u_x, v_x = net_uv(x,t)
       
       u_t = tape.gradient(u, t)[0]
       u_xx = tape.gradient(u_x, x)[0]
       
       v_t = tape.gradient(v, t)[0]
       v_xx = tape.gradient(v_x, x)[0]
        
    f_u = u_t + 0.5*v_xx + (u**2 + v**2)*v
    f_v = v_t - 0.5*u_xx - (u**2 + v**2)*u   
    
    return f_u, f_v

In [16]:

# Loss
def Loss2(u0_pred, v0_pred,u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred,u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred,f_u_pred, f_v_pred):
       loss = tf.reduce_mean(input_tensor=tf.square(u0_tf - u0_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(v0_tf - v0_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(u_lb_pred - u_ub_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(v_lb_pred - v_ub_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(u_x_lb_pred - u_x_ub_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(v_x_lb_pred - v_x_ub_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(f_u_pred)) + \
              tf.reduce_mean(input_tensor=tf.square(f_v_pred))
       return loss

In [17]:
def callback(self, xr=None):
    if self.iter % 50 == 0:
        print('It {:05d}: loss = {:10.8e}'.format(self.iter,self.current_loss))
    self.hist.append(self.current_loss)
    self.iter+=1

In [18]:
nn = NN_manual()
nn.load_weights("/kaggle/input/sec4/keras/default/1/nn2_3.weights.h5")
train_loss_record2 = []

#lr = tf.keras.optimizers.schedules.PiecewiseConstantDecay([1000,3000],[1e-2,1e-3,5e-4])
lr = 5e-4
# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)

In [19]:
lr = 5e-4
train_loss_record2 = []
train_vars = nn.trainable_variables
# Choose the optimizer
optim = tf.keras.optimizers.Adam(learning_rate=lr)
train_loss_record2 = []
for i in range(2):
    with tf.GradientTape() as t:                 
            u0_pred, v0_pred = net(q, t0_tf)
            u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred = net_uv(x_lb_tf, t_lb_tf)
            u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = net_uv(x_ub_tf, t_ub_tf)
            f_u_pred, f_v_pred = net_f_uv(x_f_tf, t_f_tf)
            loss = Loss2(u0_pred, v0_pred,u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred,u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred,f_u_pred, f_v_pred)
            grad = t.gradient(loss, train_vars)
            optim.apply_gradients(zip(grad, train_vars))
            train_loss_record2.append(loss)


In [20]:
i = 0
z2 = {}
for a in range(optim.variables.__len__()):
   
    k = str(i) 
    a = np.loadtxt('/kaggle/input/adam3v/var'+str(i)+'.csv')
    z2[k] = a
    i += 1

In [21]:
optim.load_own_variables(z2)

In [22]:
z = {}
optim.save_own_variables(z)

In [23]:
z2.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21'])

In [24]:
z.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21'])

In [25]:
k = int(20000)


epochs =  k
for iter in range(0,epochs): 
        with tf.GradientTape() as t:                 
                u0_pred, v0_pred = net(q, t0_tf)
                u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred = net_uv(x_lb_tf, t_lb_tf)
                u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = net_uv(x_ub_tf, t_ub_tf)
                f_u_pred, f_v_pred = net_f_uv(x_f_tf, t_f_tf)
                loss = Loss2(u0_pred, v0_pred,u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred,u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred,f_u_pred, f_v_pred)
                grad = t.gradient(loss, train_vars)
                optim.apply_gradients(zip(grad, train_vars))
                train_loss_record2.append(loss)
        if iter % 50 == 0:
                print('It {:05d}: loss = {:10.8e}'.format(iter,loss))



It 00000: loss = 9.93121299e-04
It 00050: loss = 2.89566116e-04
It 00100: loss = 8.25802199e-05
It 00150: loss = 5.93334516e-05
It 00200: loss = 4.72548491e-05
It 00250: loss = 3.88319495e-05
It 00300: loss = 3.25262918e-05
It 00350: loss = 2.76585670e-05
It 00400: loss = 3.07694027e-05
It 00450: loss = 2.09415812e-05
It 00500: loss = 1.84155651e-05
It 00550: loss = 1.64793018e-05
It 00600: loss = 3.49184047e-05
It 00650: loss = 1.43630732e-05
It 00700: loss = 1.27653739e-05
It 00750: loss = 1.18617836e-05
It 00800: loss = 1.14888480e-05
It 00850: loss = 1.34959600e-05
It 00900: loss = 1.03088132e-05
It 00950: loss = 9.75515104e-06
It 01000: loss = 1.88894264e-05
It 01050: loss = 8.93079596e-06
It 01100: loss = 9.63967614e-06
It 01150: loss = 8.53759138e-06
It 01200: loss = 8.05432865e-06
It 01250: loss = 1.50375694e-04
It 01300: loss = 8.55130020e-06
It 01350: loss = 7.35603862e-06
It 01400: loss = 7.09267897e-06
It 01450: loss = 9.92256901e-06
It 01500: loss = 6.99730754e-06
It 01550

In [26]:
#nn.saveweights
nn.save_weights('/kaggle/working/nn2_3.weights.h5')
np.savetxt('/kaggle/working/train_loss_record23.csv', train_loss_record2, fmt='%f')

In [27]:
#nn = neural_net(components)
"""An example of using tfp.optimizer.lbfgs_minimize to optimize a TensorFlow z.
This code shows a naive way to wrap a tf.keras.z and optimize it with the L-BFGS
optimizer from TensorFlow Probability.
Python interpreter version: 3.6.9
TensorFlow version: 2.0.0
TensorFlow Probability version: 0.8.0
NumPy version: 1.17.2
Matplotlib version: 3.1.1
"""
import numpy
import tensorflow as tf
import tensorflow_probability as tfp
from matplotlib import pyplot


def function_factory(z, loss):# train_x, train_y):
    """A factory to create a function required by tfp.optimizer.lbfgs_minimize.
    Args:
        z [in]: an instance of `tf.keras.z` or its subclasses.
        loss [in]: a function with signature loss_value = loss(pred_y, true_y).
        train_x [in]: the input part of training data.
        train_y [in]: the output part of training data.
    Returns:
        A function that has a signature of:
            loss_value, gradients = f(z_parameters).
    """

    # obtain the shapes of all trainable parameters in the z
    shapes = tf.shape_n(z.trainable_variables)
    n_tensors = len(shapes)

    # we'll use tf.dynamic_stitch and tf.dynamic_partition later, so we need to
    # prepare required information first
    count = 0
    idx = [] # stitch indices
    part = [] # partition indices

    for i, shape in enumerate(shapes):
        n = np.product(shape)
        idx.append(tf.reshape(tf.range(count, count+n, dtype=tf.int32), shape))
        part.extend([i]*n)
        count += n

    part = tf.constant(part)

    @tf.function
    def assign_new_z_parameters(params_1d):
        """A function updating the z's parameters with a 1D tf.Tensor.
        Args:
            params_1d [in]: a 1D tf.Tensor representing the z's trainable parameters.
        """

        params = tf.dynamic_partition(params_1d, part, n_tensors)
        for i, (shape, param) in enumerate(zip(shapes, params)):
            z.trainable_variables[i].assign(tf.reshape(param, shape))

    # now create a function that will be returned by this factory
    @tf.function
    def f(params_1d):
        """A function that can be used by tfp.optimizer.lbfgs_minimize.
        This function is created by function_factory.
        Args:
           params_1d [in]: a 1D tf.Tensor.
        Returns:
            A scalar loss and the gradients w.r.t. the `params_1d`.
        """
        with tf.GradientTape() as t: 
                #t.watch(train_vars)
            assign_new_z_parameters(params_1d)
            u0_pred, v0_pred,b,a = net_uv(q, t0_tf)
            u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred = net_uv(x_lb_tf, t_lb_tf)
            u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = net_uv(x_ub_tf, t_ub_tf)
            f_u_pred, f_v_pred = net_f_uv(x_f_tf, t_f_tf)
            loss_value = Loss2(u0_pred, v0_pred,u_lb_pred, v_lb_pred, u_x_lb_pred, v_x_lb_pred,u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred,f_u_pred, f_v_pred)
            grads = t.gradient(loss_value, z.trainable_variables)

        # calculate gradients and convert to 1D tf.Tensor
        #grads = tape.gradient(loss_value, z.trainable_variables)
        print(type(grads[0]))
        grads = tf.dynamic_stitch(idx, grads)
       

        # print out iteration & loss
        f.iter.assign_add(1)
        tf.print("Iter:", f.iter, "loss:", loss_value)

        # store loss value so we can retrieve later
        tf.py_function(f.history.append, inp=[loss_value], Tout=[])

        return loss_value, grads

    # store these information as members so we can use them outside the scope
    f.iter = tf.Variable(0)
    f.idx = idx
    f.part = part
    f.shapes = shapes
    f.assign_new_z_parameters = assign_new_z_parameters
    f.history = []

    return f


# use float64 by default
tf.keras.backend.set_floatx("float64")


func = function_factory(nn,Loss2)

# convert initial z parameters to a 1D tf.Tensor
init_params = tf.dynamic_stitch(func.idx, nn.trainable_variables)

# train the z with L-BFGS solver
results = tfp.optimizer.lbfgs_minimize(value_and_gradients_function=func, initial_position=init_params,max_iterations=10000,f_absolute_tolerance=1.0 * np.finfo(float).eps,max_line_search_iterations=100,num_correction_pairs=100)
#tfp.optimizer.lbfgs_minimize(
    # value_and_gradients_function=func, initial_position=init_params, max_iterations=1000)

# after training, the final optimized parameters are still in results.position
# so we have to manually put them back to the z
func.assign_new_z_parameters(results.position)
l3 =func.history 



<ipython-input-27-6015997f0176>:108: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  func = function_factory(nn,Loss2)


<class 'tensorflow.python.framework.ops.SymbolicTensor'>
Iter: 1 loss: 7.32094122e-06
Iter: 2 loss: 0.108545162
Iter: 3 loss: 4.12513873e-06
Iter: 4 loss: 3.47259765e-06
Iter: 5 loss: 7.79828952e-06
Iter: 6 loss: 3.40277211e-06
Iter: 7 loss: 3.10423866e-06
Iter: 8 loss: 3.07279652e-06
Iter: 9 loss: 2.8538052e-06
Iter: 10 loss: 2.78933476e-06
Iter: 11 loss: 2.82280757e-06
Iter: 12 loss: 2.74661033e-06
Iter: 13 loss: 2.72063789e-06
Iter: 14 loss: 2.72058378e-06
Iter: 15 loss: 2.6942696e-06
Iter: 16 loss: 2.7653366e-06
Iter: 17 loss: 2.68565327e-06
Iter: 18 loss: 2.6783473e-06
Iter: 19 loss: 2.70208238e-06
Iter: 20 loss: 2.67631981e-06
Iter: 21 loss: 2.6736443e-06
Iter: 22 loss: 2.67091605e-06
Iter: 23 loss: 2.67032e-06
Iter: 24 loss: 2.66744155e-06
Iter: 25 loss: 2.66072425e-06
Iter: 26 loss: 2.74138824e-06
Iter: 27 loss: 2.66022e-06
Iter: 28 loss: 2.65547465e-06
Iter: 29 loss: 2.65378367e-06
Iter: 30 loss: 2.65119252e-06
Iter: 31 loss: 2.64796427e-06
Iter: 32 loss: 2.64790469e-06
Iter: 

optim.variables

In [28]:
store = {}
optim.save_own_variables(store)
u = []

i = 0
for a in store.values():
    
    u.append(a)

    k = str(i)
    if np.shape(a) == ():
        a = [a]

    
    np.savetxt('/kaggle/working/var'+str(i)+'.csv',a)
    i += 1

In [29]:
#nn.saveweights
nn.save_weights('/kaggle/working/nn2_4.weights.h5')


In [30]:

np.savetxt('/kaggle/working/l3.csv', l3, fmt='%f')

In [ ]:
u_v = nn.predict(X_star)
u_pred, v_pred = u_v[:,0:1],u_v[:,1:2]
f_u_pred,f_v_pred = net_f_uv(tf.constant(X_star[:,0:1],dtype="float32"),tf.constant(X_star[:,1:2],dtype="float32"))
u_v = nn.predict(X_star)
#u_pred, v_pred = u_v[:,0:1],u_v[:,1:2]
#f_u_pred,f_v_pred = net_f_uv(tf.constant(X_star[:,0:1],float64),tf.constant(X_star[:,1:2]))
h_pred = np.sqrt(u_pred**2 + v_pred**2)
        
error_u = np.linalg.norm(u_star-u_pred,2)/np.linalg.norm(u_star,2)
error_v = np.linalg.norm(v_star-v_pred,2)/np.linalg.norm(v_star,2)
error_h = np.linalg.norm(h_star-h_pred,2)/np.linalg.norm(h_star,2)
print('Error u: %e' % (error_u))
print('Error v: %e' % (error_v))
print('Error h: %e' % (error_h))


U_pred = griddata(X_star, u_pred.flatten(), (X, T), method='cubic')
V_pred = griddata(X_star, v_pred.flatten(), (X, T), method='cubic')
H_pred = griddata(X_star, h_pred.flatten(), (X, T), method='cubic')

FU_pred = griddata(X_star, np.array(f_u_pred).flatten(), (X, T), method='cubic')
FV_pred = griddata(X_star, np.array(f_v_pred).flatten(), (X, T), method='cubic')
gs1 = gridspec.GridSpec(1, 3)
gs1.update(top=1-1/3, bottom=0, left=0.1, right=0.9, wspace=0.5)

ax = plt.subplot(gs1[0, 0])
ax.plot(x,Exact_h[:,75], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,H_pred[75,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$|h(t,x)|$')    
#ax.set_title('$t = %.2f$' % (t[75]), fontsize = 10)
ax.axis('square')
ax.set_xlim([-5.1,5.1])
ax.set_ylim([-0.1,5.1])

ax = plt.subplot(gs1[0, 1])
ax.plot(x,Exact_h[:,100], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,H_pred[100,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$|h(t,x)|$')
ax.axis('square')
ax.set_xlim([-5.1,5.1])
ax.set_ylim([-0.1,5.1])
#ax.set_title('$t = %.2f$' % (t[100]), fontsize = 10)
ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.8), ncol=5, frameon=False)

ax = plt.subplot(gs1[0, 2])
ax.plot(x,Exact_h[:,125], 'b-', linewidth = 2, label = 'Exact')       
ax.plot(x,H_pred[125,:], 'r--', linewidth = 2, label = 'Prediction')
ax.set_xlabel('$x$')
ax.set_ylabel('$|h(t,x)|$')
ax.axis('square')
ax.set_xlim([-5.1,5.1])
ax.set_ylim([-0.1,5.1])    
#ax.set_title('$t = %.2f$' % (t[125]), fontsize = 10)

In [31]:
#np.savetxt('/kaggle/working/optimizer.txt', optim.variables)
print(optim.variables)



[<Variable path=adam/iteration, shape=(), dtype=int64, value=110000>, <Variable path=adam/learning_rate, shape=(), dtype=float32, value=0.0005000000237487257>, <Variable path=adam/sequential_dense_10_kernel_momentum, shape=(2, 100), dtype=float32, value=[[ 3.28252281e-05  9.63115308e-05  2.50287439e-06 -6.80262383e-06
   1.02628746e-06  1.46050206e-05  4.91247338e-05 -1.31520301e-05
  -1.30934772e-04  5.09803112e-06 -1.02543083e-04 -1.38909127e-05
  -8.38030155e-06 -1.10134361e-05 -1.27672363e-04  2.34871604e-05
   2.98500163e-05 -1.48473609e-05  2.66538227e-05  5.87035265e-06
   9.14622524e-06  2.46833770e-06 -6.30258637e-06 -6.83240796e-06
  -5.03954689e-05 -1.55812195e-05  3.42585408e-05 -1.39101280e-06
   1.65819602e-05 -1.04586052e-05  3.13399614e-05  3.76259982e-06
   2.34197523e-05  2.09244990e-05  5.98834413e-06 -2.01980129e-05
   2.27919736e-05 -1.93855121e-05 -1.73297485e-05 -1.81546829e-06
  -1.48034324e-05 -1.15945504e-05  7.11422326e-05  6.50011680e-06
   5.69777330e-06 -3

Loss